In [1]:
from data.graph_loader import GraphLoader
from data.gtfs_loader import GTFSLoader
from core.routing import RouteService
import pandas as pd
import geopandas as gpd
import osmnx as ox
import networkx as nx

def openMap(m):
    html = "map.html"
    m.save(html)

    import webbrowser
    webbrowser.open(html)

In [2]:
gtfs_loader = GTFSLoader()
transit_df = gtfs_loader.load_transit_dataframe("data/gtfs")
stops_df = gtfs_loader.load_stops_dataframe("data/gtfs", transit_df)



Loading transit DataFrame from data/gtfs\transit_df.pkl
Loading stops DataFrame from data/gtfs\stops_df.pkl


In [3]:
graph_loader = GraphLoader()
graph_walk = graph_loader.create_graph_walk("data/graphs/ZMG_walk.pkl", "data/osm/ZMG_enclosure_2km.geojson")
graph_transit = graph_loader.create_graph_transit("data/graphs/ZMG_transit.pkl", stops_df)

Loading walking graph from data/graphs/ZMG_walk.pkl
Loading transit graph from data/graphs/ZMG_transit_300.pkl


In [4]:
route_service = RouteService(graph_walk, graph_transit, stops_df, transit_df)

In [9]:
import random as rd
from shapely.geometry import Point

#choose randomly a node from the largest connected component
def random_conected_walking_nodes():

    connected = False
    attempts = 0

    while not connected:
        # Pick from largest connected component
        start = rd.choice(list(graph_walk.nodes))
        destination = rd.choice(list(graph_walk.nodes))
        
        # Since they're both in the same connected component, they should be connected
        connected = nx.has_path(graph_walk, source=start, target=destination)
        attempts += 1
        print(f"Attempt {attempts}")
        if attempts > 100:  # Much fewer attempts needed now
            raise Exception("Failed to find connected nodes after 100 attempts")
            
    return start, destination

walk_node_start, walk_node_destination = random_conected_walking_nodes()

Attempt 1


In [12]:
import folium

start = Point(graph_walk.nodes[walk_node_start]['x'], graph_walk.nodes[walk_node_start]['y'])
destination = Point(graph_walk.nodes[walk_node_destination]['x'], graph_walk.nodes[walk_node_destination]['y'])

df, total_time, m_comb, path_transit = route_service.route_combined(start, destination)
# Add layer control
folium.LayerControl().add_to(m_comb)

openMap(m_comb)



In [11]:
m = route_service.create_map(df)
openMap(m)